<a href="https://colab.research.google.com/github/rabbi123080/Prediction-of-Liver-Disorders-using-Machine-Learning-Algorithms-A-Comparative-Study/blob/main/ILPD_ICAICT_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ilpd.csv to ilpd.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['ilpd.csv']))
df.dty

V1         int64
V2        object
V3       float64
V4       float64
V5         int64
V6         int64
V7         int64
V8       float64
V9       float64
V10      float64
Class      int64
dtype: object

In [ ]:
X = df.drop('Class', axis = 1)
y = df['Class']

In [ ]:
sex = pd.get_dummies(X['V2'], drop_first=True)
X = pd.concat([X,sex], axis=1)
X.drop(['V2'], axis=1, inplace=True)

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE
counter = Counter(y)
print(counter)

Counter({1: 416, 2: 167})


In [ ]:
from imblearn.over_sampling import RandomOverSampler
over = RandomOverSampler()
X, y = over.fit_resample(X, y)
counter = Counter(y)
print(counter)

Counter({1: 416, 2: 416})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [ ]:
from sklearn.preprocessing import RobustScaler

sc_X = RobustScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.fit_transform(X_test)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
rfc = AdaBoostClassifier(base_estimator=RandomForestClassifier(), n_estimators=30)

rfc.fit(X_train, y_train)

rfc_pred = rfc.predict(X_test)
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(y_test, rfc_pred))
print(classification_report(y_test, rfc_pred))

[[64 15]
 [11 77]]
              precision    recall  f1-score   support

           1       0.85      0.81      0.83        79
           2       0.84      0.88      0.86        88

    accuracy                           0.84       167
   macro avg       0.85      0.84      0.84       167
weighted avg       0.84      0.84      0.84       167



In [ ]:
from sklearn.model_selection import RandomizedSearchCV
parameters = {    
 'learning_rate': [0.01, 0.02, 0.05, 0.1, 0.3, 0.5, 1],
 'n_estimators': [20, 30, 50, 75, 100, 110],   
 'base_estimator__n_estimators': [20, 30, 50, 75, 100, 110],
  'base_estimator__max_depth': [10, 15, 20, 30, 40],
   
 }

In [ ]:
grid=RandomizedSearchCV(AdaBoostClassifier(base_estimator=RandomForestClassifier()), parameters)

In [ ]:
grid.fit(X_train, y_train)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=AdaBoostClassifier(algorithm='SAMME.R',
                                                base_estimator=RandomForestClassifier(bootstrap=True,
                                                                                      ccp_alpha=0.0,
                                                                                      class_weight=None,
                                                                                      criterion='gini',
                                                                                      max_depth=None,
                                                                                      max_features='auto',
                                                                                      max_leaf_nodes=None,
                                                                                      max_samples=None,
                                                      

In [ ]:
grid.best_params_

{'base_estimator__max_depth': 30,
 'base_estimator__n_estimators': 20,
 'learning_rate': 0.5,
 'n_estimators': 30}

In [ ]:
grid.best_estimator_

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.5,
                   n_estimators=110, random_state=None)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
accuracy = cross_val_score(AdaBoostClassifier(RandomForestClassifier(n_estimators=20, max_depth=30), n_estimators=30, learning_rate=0.5), X, y, cv = 10, scoring='accuracy')
print(accuracy)

[0.88095238 0.85714286 0.87951807 0.81927711 0.85542169 0.87951807
 0.8313253  0.86746988 0.95180723 0.95180723]


In [ ]:
print(accuracy.mean())

0.8774239816408491


In [ ]:
f1 = accuracy = cross_val_score(grid.best_estimator_, X, y, cv = 10, scoring='f1')
print(f1)

[0.88607595 0.83544304 0.825      0.86419753 0.74626866 0.7826087
 0.71875    0.7761194  0.92105263 0.96202532]


In [ ]:
print(f1.mean())

0.8317541221594281


In [ ]:
roc = accuracy = cross_val_score(grid.best_estimator_, X, y, cv = 20, scoring='roc_auc')
print(roc)

[0.98639456 0.97845805 0.93764172 0.94444444 0.94104308 0.96031746
 0.9478458  0.9047619  0.98412698 0.99319728 1.         0.98866213
 0.96071429 0.98809524 0.9952381  1.         0.98809524 1.
 0.9952381  0.99880952]


In [ ]:
print(roc.mean())

0.9746541950113377
